<a href="https://colab.research.google.com/github/Pinkysol/Learning/blob/main/FAST_API_MNIST_REQUEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В домашней работе необходимо с помощью **FastAPI** реализовать **REST API**:

1. На 3 балла. Твой **REST API** - это список покупок и содержит поля: название товара, группа товара (например, электроника или продовольствие), цена, единица измерения, количество. Также необходимо реализовать метод, который возвращает список - расходы по каждой группе товаров и сумму всех покупок. Также необходимо с помощью библиотеки *requests* продемонстрировать запросами к **REST API**, как работает ваш веб-сервис. Это задание можно сравнить с "покрытием тестами" вашего API. Нечто похожее делают тестировщики в ИТ-компаниях. Вам необходимо покрыть запросами все методы, которые вы реализуете на веб-сервере.
2. **На 4 балла**. Вам необходимо сделать красивую документацию для твоего **REST API** с подробным описанием. Для этого тебе придется обратиться к документации:
  
  https://fastapi.tiangolo.com/ru/tutorial/metadata/
  
  https://fastapi.tiangolo.com/ru/tutorial/path-operation-configuration/#response-description

3. **На 5 баллов**. Творческое задание. **REST API** можно использовать для взаимодействия с вашей моделью нейронной сети. Вы уже знаете, что можно обучить модель, а лучший результат выгрузить для дальнейшего использования. Для получения 5 баллов необходимо обучить свою модель, загрузить ее в **Colab**. Задача может быть любой: регрессии, классификации, входными данными могут быть картинки или текстовые данные. С помощью **REST API** обеспечьте взаимодействие с моделью. Это полностью творческое задание!

In [1]:
!pip install autokeras
!pip install uvicorn
!pip install fastapi
!pip install uvicorn
!pip install python-multipart

In [2]:
# Импортируем фреймворк Keras
import keras
from keras.datasets import mnist
# Библиотека AutoML autokeras

import autokeras as ak
# Утилита для расщепления выборки
from sklearn.model_selection import train_test_split

# Загрузка датасета https://keras.io/api/datasets/mnist/
(x_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_val, y_train, y_val = train_test_split(x_train,
  y_train, test_size=0.2, stratify=y_train, random_state=42)

# Сборка модели autokeras
clf = ak.ImageClassifier(overwrite=True, max_trials=1)
result_training = clf.fit(X_train, y_train, epochs = 1, validation_data = (X_val, y_val))
# Сохранение лучшей модели
model = clf.export_model()
model.save("model_autokeras.keras")

Trial 1 Complete [00h 00m 24s]
val_loss: 0.06417687237262726

Best val_loss So Far: 0.06417687237262726
Total elapsed time: 00h 00m 24s
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 13s 5ms/step - accuracy: 0.8895 - loss: 0.3445 - val_accuracy: 0.9826 - val_loss: 0.0612


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [3]:
%%writefile main.py
import io
import pickle
import numpy as np
import PIL.Image
import PIL.ImageOps
from keras.models import load_model
from fastapi import FastAPI, File, UploadFile
from fastapi.middleware.cors import CORSMiddleware
import autokeras as ak

app = FastAPI()
# Загрузка обученной модели на датасете mnist
model = load_model(
    "model_autokeras.keras", custom_objects=ak.CUSTOM_OBJECTS
)

# Описание действий сервера при загрузке изображения
@app.post("/number_predict")
async def predict_image(file: UploadFile = File(...)):
    # Чтение файла
    contents = await file.read()
    # Открываем файл с помощью библиотеки по работе с изображениями
    pil_image = PIL.Image.open(io.BytesIO(contents))
    # Файл приходит с отсутствующими пикселями из-за его представления в байтах, растягиваем до нужного размера
    pil_image = pil_image.resize((28, 28), PIL.Image.Resampling.LANCZOS)
    # Преобразуем файл в массив numpy
    img_array = np.array(pil_image)
    # Добавляем измерение для предсказания (1,28,28)
    img_array = np.expand_dims(img_array, axis=0)
    # Предсказываем число по изображению
    prediction = model.predict(img_array)
    # В результате предсказания выбираем индекс значения с максимальным значением
    predicted_value = prediction[0].argmax(axis=0)
    return {"prediction": str(predicted_value)}

Overwriting main.py


In [4]:
!nohup uvicorn main:app --reload &

nohup: appending output to 'nohup.out'


In [5]:
!cat nohup.out

INFO:     Will watch for changes in these directories: ['/content']
ERROR:    [Errno 98] Address already in use
INFO:     Will watch for changes in these directories: ['/content']
ERROR:    [Errno 98] Address already in use
INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [5926] using StatReload
2025-05-11 01:47:13.073204: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746928033.095612    5928 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746928033.105750    5928 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-11 0

In [12]:
import requests
import io
from keras.models import load_model
import PIL.Image

# Скачиваем изображение цифры 0
!wget https://storage.yandexcloud.net/quiz-bucket-pinkysoul/img_2.jpg

# Открытие файла
with open("img_2.jpg", 'rb') as f:
    img_data = f.read()

# Отправка файла запросом Post
files = {'file': img_data}
r = requests.post('http://127.0.0.1:8000/number_predict', files=files)

print('Результат запроса: ', r)
number = r.json()
print('Результат предсказания: ', number['prediction'])

--2025-05-11 02:18:26--  https://storage.yandexcloud.net/quiz-bucket-pinkysoul/img_2.jpg
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 615 [image/jpeg]
Saving to: ‘img_2.jpg.1’

img_2.jpg.1         100%[===================>]     615  --.-KB/s    in 0s      

2025-05-11 02:18:28 (300 MB/s) - ‘img_2.jpg.1’ saved [615/615]

Результат запроса:  <Response [200]>
Результат предсказания:  0


In [7]:
# Вывод отправленного изображения
pil_image = PIL.Image.open(io.BytesIO(img_data))
pil_image

Как видно выше, на сервере был предсказана цифра 0, она же и была отправлена

In [10]:
!ssh -o "StrictHostKeyChecking no" -R 80:localhost:8000 nokey@localhost.run


Welcome to localhost.run!

Follow your favourite reverse tunnel at [https://twitter.com/localhost_run].

To set up and manage custom domains go to https://admin.localhost.run/

More details on custom domains (and how to enable subdomains of your custom
domain) at https://localhost.run/docs/custom-domains

If you get a permission denied error check the faq for how to connect with a key or
create a free tunnel without a key at [http://localhost:3000/docs/faq#generating-an-ssh-key].

To explore using localhost.run visit the documentation site:
https://localhost.run/docs/


** your connection id is df110307-2a76-4adc-aad7-0009f1679732, please mention it if you send me a message about an issue. **

authenticated as anonymous user
ee41e51a7dbb57.lhr.life tunneled with tls termination, https://ee41e51a7dbb57.lhr.life
create an account and add your key for a longer lasting domain name. see https://localhost.run/docs/forever-free/ for more information.
Open your tunnel address on your mobile w